# Prep Work incl. importing libraries, combining datasets

Datasets (no. of songs): <br>
2014 = 232 <br>
2015 = 370 <br>
2016 = 582 <br>
2017 = 1294 <br>
2018 = 2516 <br>
TOTAL = 4994

In [3]:
# mass imports
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import time
songs_combine = pd.read_csv("song_features/spotifybillboard_hitmiss_2014to18.csv")

# #2 K Nearest Neighbors
Some potential: Model with optimal k has accuracy of around 61.7%...slightly better than flipping a coin?

In [4]:
features_all=['duration','loudness', 'tempo','tempo_confidence','time_signature_confidence',
                   'time_signature','key_confidence','mode_confidence',
                   'key', 'mode', 'acousticness', 'danceability', 'energy',
                   'instrumentalness', 'liveness', 'speechiness', 'valence']
features_all.sort()
print(len(features_all))
features_all

17


['acousticness',
 'danceability',
 'duration',
 'energy',
 'instrumentalness',
 'key',
 'key_confidence',
 'liveness',
 'loudness',
 'mode',
 'mode_confidence',
 'speechiness',
 'tempo',
 'tempo_confidence',
 'time_signature',
 'time_signature_confidence',
 'valence']

In [5]:
y = songs_combine[['hitmiss_billboard']]

In [6]:
def model_score(x):
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, test_size = 0.2, random_state = 1)
    # for KNeighborsClassifier, need to identify best k neighbors model by checking which k gives the highest validation accuracy a.k.a. score

    # create empty list to store scores for each k
    scores = []

    # for each k from 1 to 300, create a model and add its score to the "scores" list
    for i in range(1,301):
        classifier = KNeighborsClassifier(n_neighbors = i)
        classifier.fit(x_train,y_train.values.ravel())
        score = classifier.score(x_test,y_test)
        scores.append(score)
    
    # create a list of k from 1 to 300
    k_lib = range(1,301)

    # combine lists of k and scores into dataframe "k_scores"
    k_scores = pd.DataFrame({"k":k_lib, "Scores":scores})

    # in "k_scores", find index with maximum score, then find corresponding k (in this case 61)
    max_score_index = k_scores["Scores"].idxmax(axis = 0)
    max_score_k = k_scores["k"][max_score_index]

    # create a model using best k (in this case 107)
    classifier = KNeighborsClassifier(n_neighbors = 61)
    classifier.fit(x_train,y_train.values.ravel())
    return (classifier.score(x_test,y_test))

In [11]:
features_combination=[]
models_score_combination=[]
from itertools import combinations
for i in range(16,18):
    comb = combinations(features_all,i)
    for i in list(comb):
        features_combination.append(i)
        list_i=list(i)    
        models_score_combination.append(model_score(songs_combine[list_i]))

In [12]:
print(len(features_combination))
print(len(models_score_combination))

18
18


In [13]:
max(models_score_combination)

0.7378277153558053

In [14]:
selected_features=list(features_combination[models_score_combination.index(max(models_score_combination))])
selected_features

['acousticness',
 'danceability',
 'duration',
 'energy',
 'instrumentalness',
 'key',
 'key_confidence',
 'liveness',
 'loudness',
 'mode',
 'mode_confidence',
 'speechiness',
 'tempo',
 'tempo_confidence',
 'time_signature',
 'time_signature_confidence']

In [15]:
X=songs_combine[selected_features]